In [33]:
import logging
import os.path
from datetime import datetime

import cmocean
import gcsfs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cpf
import seaborn as sns
import pandas as pd
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely import geometry
import warnings
import os
from google.cloud import storage
from zarr.convenience import consolidate_metadata
import global_land_mask
from IPython.display import display, HTML
warnings.filterwarnings('ignore')
plt.style.use('mpl20')

#### Read the locations of seabird colonies
This file created by Sarah Ann (June 2022) and can be found under gcs: `Commons/NSF_seabirds_stratificiation`

In [34]:
infile="../Seabird_sites.xlsx"
df = pd.read_excel(infile, header=[0])
print(df.head())

# Google Cloud client
ACTEA_bucket = "actea-shared"
fs = gcsfs.GCSFileSystem(project="downscale")
storage_client = storage.Client()
bucket = storage_client.bucket(ACTEA_bucket)

        Site     Location   Latitude  Longitude
0      Adams  New Zealand -50.902700  166.00180
1  Admiralty   Antarctica -62.177800  -58.44470
2     Aiktak          USA  54.188600 -164.84000
3   Alcatraz          USA  37.826670 -122.42333
4  Amsterdam       France -37.816667   77.53333


DefaultCredentialsError: File /Users/trondkr/PycharmProjects/Actea/Downscale-credentials/acteatmac_04032022_downscale-5f0121b6020f.json was not found.

In [ ]:
# Select specific months January through April
def is_jfma(month):
    return (month >= 1) & (month <= 4)

def organize_dataset(self, variable_id, st_lon, st_lat,):

    baseURL = self.get_dataset_url(parameter=variable_id)
    print(f"Organizing GLORYS data at path {baseURL}")

    mapper = fs.get_mapper(
        self.get_dataset_url(parameter=variable_id)
    )

    consolidate_metadata(mapper)
    d = xr.open_zarr(mapper, consolidated=False)

    # To get consistent naming convention between GLORYS and CMIP6 we need to
    # change the name of the GLORYS variable from nppv to intpp.
    if variable_id=="intpp":
        d=d.rename({"nppv":"intpp"})

    d["mask"] = create_land_ocean_mask(d)
    ds_masked = d.where(d.mask == 1, drop=True)

    lon_index, lat_index, distance_km = get_index_closest_point(
        st_lon, st_lat, ds_masked.lon, ds_masked.lat, ds_masked[variable_id].isel(time=0)
    )

    # ds_interp = ds_masked.interp(lat=lat, lon=lon, method="linear")
    ds_interp = ds_masked.sel(
        lat=ds_masked.lat[lat_index].values,
        lon=ds_masked.lon[lon_index].values,
        method="nearest",
    )

    # Limit to the period of interest
   # ds_interp = ds_interp.sel(time=slice(start_time, end_time))

    # Find the index for the bottom values, unless we are looking at chloroophyll
    if variable_id in ["chl"]:
        ind = 0
    else:
        ind = find_bottom_index(ds_interp, variable_id)
    sel_depth = ds_interp.depth.values[ind]
    print(
        "Values will be extracted for depth {:3.1f} m at lat: {} lon: {}".format(
            float(sel_depth), ds_interp.lat.values, ds_interp.lon.values
        )
    )
     # Select time-constraint and limit to January through April
    seasonal_ds = (
        ds_interp.sel(depth=sel_depth)
        .sel(time=is_jfma(ds_interp["time.month"]))
        .resample(time="A")
        .mean()
    )
    seasonal_ds["distance_to_original_point"] = distance_km

    return seasonal_ds, d


def get_dataset_url(self, parameter):
    try:
        return {
            "o2": f"gs://{self.config.ACTEA_bucket}/zarr/o2/",
            "no3": f"gs://{self.config.ACTEA_bucket}/zarr/no3/",
            "si": f"gs://{self.config.ACTEA_bucket}/zarr/si/",
            "so": f"gs://{self.config.ACTEA_bucket}/zarr/so/",
            "thetao": f"gs://{self.config.ACTEA_bucket}/zarr/thetao/",
            "chl": f"gs://{self.config.ACTEA_bucket}/zarr/chl/",
            "intpp": f"gs://{self.config.ACTEA_bucket}/zarr/nppv/",
            "uo": f"gs://{self.config.ACTEA_bucket}/zarr/uo/",
            "vo": f"gs://{self.config.ACTEA_bucket}/zarr/vo/",
            "ph": f"gs://{self.config.ACTEA_bucket}/zarr/ph/",
            "spco2": f"gs://{self.config.ACTEA_bucket}/zarr/spco2/",
            "siconc": f"gs://{self.config.ACTEA_bucket}/zarr/siconc/",
            "sithick": f"gs://{self.config.ACTEA_bucket}/zarr/sithick/",
        }[parameter]
    except Exception as e:
        return e

# Return the index of deepest depth where we have values at this station
def find_bottom_index(ds, var_name):
    ds_one = ds.isel(time=0)
    vals = ds_one[var_name].values

    ind = np.where(np.isnan(vals))[0][0] - 1
    return ind

def create_land_ocean_mask(self, ACTEA_grid: xr.Dataset) -> xr.DataArray:
    """
    Function that creates land mask based on its longitude - latitude.
    Returns a DataArray to be included in a Dataset and used for extrapolation
    in xesmf.
    """
    print("Running create_land_ocean_mask for")

    if ACTEA_grid.lon.ndim == 1:
        lon = ACTEA_grid.lon.values
        lat = ACTEA_grid.lat.values
    elif ACTEA_grid.lon.ndim == 2:
        lon = ACTEA_grid.lon[0, :].values
        lat = ACTEA_grid.lat[:, 0].values
    else:
        raise Exception(
            "Unable to understand dimensions of longitude/latitude: {}".format(
                ACTEA_grid.lon.ndim
            )
        )

    lon_180 = xr.where(lon > 180, lon - 360, lon)
    lon_grid, lat_grid = np.meshgrid(lon_180, lat)

    mask_data = global_land_mask.globe.is_ocean(lat_grid, lon_grid)

    return xr.DataArray(
        mask_data, coords={"lat": lat, "lon": lon}, dims=["lat", "lon"]
    ).astype(int)

def create_timeseries_plot(dfs, var_name, stations):
    sns.set_style("ticks")
    sns.set_palette("tab10")

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)

    sns.lineplot(
        ax=ax,
        data=dfs,
        x=dfs.index,
        y=dfs[var_name],
        hue=dfs["station"],
        ci=95,
        alpha=0.95,
    )

    plotfile = "../oceanography/Spies_EBS/timeseries_{}.png".format(var_name)
    plt.savefig(plotfile, dpi=200)
    print("Saved map to {}".format(plotfile))
    plt.show()


def get_index_closest_point(
    st_lon: np.float,
    st_lat: np.float,
    lon: xr.DataArray,
    lat: xr.DataArray,
    da: xr.DataArray,
):

    """
    Method that finds the n number of stations closest to a st_lon, st_lat point
    and checks to see if the values only contain nan (land) or if this is a valid
    ocean station.
    """
    from sklearn.neighbors import BallTree

    lons, lats = np.meshgrid(lon.values, lat.values)
    df = pd.DataFrame()
    df["lat"] = lats.ravel()
    df["lon"] = lons.ravel()

    # Setup Balltree using df as reference dataset
    # Use Haversine calculate distance between points on the earth from lat/long
    # haversine - https://pypi.org/project/haversine/
    tree = BallTree(np.deg2rad(df[["lat", "lon"]].values), metric="haversine")

    # Station
    df_st = pd.DataFrame()
    df_st["lat"] = st_lat
    df_st["lon"] = st_lon

    # Find closest point in reference dataset for each in df_other
    # use k = 3 for 3 closest neighbors
    distances, indices = tree.query(np.deg2rad(np.c_[st_lat, st_lon]), k=10)

    r_km = 6371  # multiplier to convert to km (from unit distance)
    for d, ind in zip(distances, indices):
        print("Found closest matches:")
        for i, index in enumerate(ind):

            print(
                "Checking validity: \t{:3.2f} {:3.2f} with distance {:.4f} km".format(
                    df["lat"][index], df["lon"][index], d[i] * r_km
                )
            )
            lon_index = np.where(lon == df["lon"][index])[0]
            lat_index = np.where(lat == df["lat"][index])[0]

            if not np.isnan(da[:, lat_index, lon_index]).all():
                print("FOUND:")
                print(
                    "lon found at index {} lon {}".format(
                        lon_index[0], lon[lon_index[0]].values
                    )
                )
                print(
                    "lat found at index {} lat {}".format(
                        lat_index[0], lat[lat_index[0]].values
                    )
                )

                return lon_index[0], lat_index[0], d[i] * r_km


def get_etopo1():
    etopo1_filename = "../oceanography/ETOPO1/ETOPO1_Ice_g_gmt4.grd"
    ds_topo = xr.open_dataset(etopo1_filename)
    topo = ds_topo.z
    ds_topo.close()

    topo2 = topo.sel(y=slice(42, 66), x=slice(-180, -118))
    x = topo2.x  # 21601
    y = topo2.y  # 10801
    X, Y = np.meshgrid(x, y)
    return X, Y, topo2


def level_colormap(levels, cmap=None):
    """Make a colormap based on an increasing sequence of levels"""

    # Start with an existing colormap
    if cmap == None:
        cmap = pl.get_cmap()

    # Spread the colours maximally
    nlev = len(levels)
    S = np.arange(nlev, dtype="float") / (nlev - 1)
    A = cmap(S)

    # Normalize the levels to interval [0,1]
    levels = np.array(levels, dtype="float")
    L = (levels - levels[0]) / (levels[-1] - levels[0])

    # Make the colour dictionary
    R = [(L[i], A[i, 0], A[i, 0]) for i in range(nlev)]
    G = [(L[i], A[i, 1], A[i, 1]) for i in range(nlev)]
    B = [(L[i], A[i, 2], A[i, 2]) for i in range(nlev)]
    cdict = dict(red=tuple(R), green=tuple(G), blue=tuple(B))

    # Use
    return colors.LinearSegmentedColormap("%s_levels" % cmap.name, cdict, 256)


def create_map(ds, stations):

    projection = ccrs.PlateCarree(central_longitude=-180)
    ax = plt.figure(figsize=(16, 10)).gca(projection=projection)
    ax.coastlines(resolution="10m", linewidth=0.6, color="black", alpha=0.8, zorder=10)
    ax.add_feature(cpf.BORDERS, linestyle=":", alpha=0.4)
    ax.add_feature(cpf.LAND, color="lightgrey", zorder=9)
    ax.set_extent([-180, -120, 46, 62])

    xticks = np.linspace(-180, -120, 5)
    yticks = np.linspace(46, 62, 5)

    ax.set_xticks(xticks, crs=cartopy.crs.PlateCarree())
    ax.set_yticks(yticks, crs=cartopy.crs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    X, Y, topo2 = get_etopo1()

    plt.grid(True, zorder=0, alpha=0.5)
    levels = [
        -4500,
        -3500,
        -3000,
        -2500,
        -2000,
        -1500,
        -1000,
        -500,
        -200,
        -175,
        -150,
        -125,
        -100,
        -75,
        -50,
        -40,
        -30,
        -20,
        -10,
    ]

    cm = level_colormap(levels, cmap=cmocean.cm.deep_r)
    cs3 = ax.contourf(
        X,
        Y,
        topo2,
        cmap=cm,
        levels=levels,
        zorder=3,
        alpha=1.0,
        extend="both",
        transform=ccrs.PlateCarree(),
    )

    for station_key in stations:
        station = stations[station_key]
        lat = station["lat"]
        lon = station["lon"]
        print("Scatter", station_key, lat, lon)
        ax.scatter(lon, lat, 50, zorder=30, c="red", transform=ccrs.PlateCarree())
        plotfile = "../oceanography/Spies_EBS/map_stations.png"
    plt.savefig(plotfile, dpi=200)
    print("Saved map to {}".format(plotfile))
    plt.show()

In [ ]:
lonmin=-180
lonmax=180
latmin=-90
latmax=90

palette = sns.color_palette("bright")
ax = plt.figure(figsize=(12,12)).gca(projection=cartopy.crs.PlateCarree(central_longitude=-180))
ax.coastlines(resolution="110m", linewidth=0.6, color="black", alpha=0.8, zorder=0)
#ax.add_feature(cpf.BORDERS, linestyle=':',alpha=0.6)
ax.add_feature(cpf.LAND, color="lightgrey")
ax.set_extent([lonmin, lonmax, latmin, latmax])

# plot the lat lon labels
# https://scitools.org.uk/cartopy/docs/v0.15/examples/tick_labels.html
# https://stackoverflow.com/questions/49956355/adding-gridlines-using-cartopy
xticks = np.linspace(lonmin, lonmax, 5)
yticks = np.linspace(latmin, latmax, 5)

ax.set_xticks(xticks, crs=cartopy.crs.PlateCarree())
ax.set_yticks(yticks, crs=cartopy.crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
#palette = iter(sns.color_palette("bright")) #(len(regions)))
sns.set_palette(sns.color_palette("bright"))
radius = 10000  # in m

plt.scatter(x=df.Longitude, y=df.Latitude,
            color="dodgerblue",
            s=1,
            alpha=0.5,
            transform=ccrs.PlateCarree()) ## Important

for index, row in df.iterrows():
    center = geometry.Point(float(row.Longitude), float(row.Latitude)).buffer(5)


    ax.add_geometries([center], crs=cartopy.crs.PlateCarree(),
                          alpha=0.4, linewidth=0.0)

 #   ax.text(x,y, region_i, color='k', size=14,
 #               ha='left', va='top', transform=ccrs.PlateCarree())
if not os.path.exists("Figures"): os.makedirs("Figures", exist_ok=True)
plt.savefig("Figures/Map_seabird_colonies_NSF.png", dpi=300)
plt.show()

var_names = ["thetao", "so", "chl", "uo", "vo"]
start_time = "2003-01-01"
end_time = "2017-12-31"

first = True
df_all_vars_stations = []
stations={}

for var_name in var_names:
    dfs_all = {}
    dfs = []
    for index, row in df.iterrows():

        st_lat = float(row.Latitude)
        st_lon = float(row.Longitude)

        print("Station: {} - ({},{}) ".format(df.Site, st_lat, st_lon))
        ds_seasonal, ds = organize_dataset(
            var_name, st_lat, st_lon, start_time, end_time
        )
        df = ds_seasonal.to_dataframe()
       # df["station"] = station_key
        df.reset_index()

        dfs.append(df)

        if first:
            create_map(ds, stations)
            first = False
    dfs_all = pd.concat(dfs)
    df_all_vars_stations.append(dfs_all)

